In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
%%capture
!pip install netcdf4
!pip install geopandas

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.colors as colors
import glob as glb

In [ ]:
%cd /content/MyDrive/MyDrive/2023-1/3. Meteo fisica/Datos-rayos

/content/MyDrive/MyDrive/2023-1/3. Meteo fisica/Datos-rayos


In [ ]:
files = sorted(glb.glob("*dat"))
archivos=[]
for file in files:
   leer=pd.read_table(file, sep="\s+",header=None)
   leer.rename(columns={0:"year",1:"month",2:"day",3:"hour",4:"minute",
                        5:"second",7:"Y",8:"X",11:"Control",
                        12:"Polaridad"}, inplace=True )
   columnas=["year","month","day","hour","minute","second","Y","X",
             "Control","Polaridad"]
   df = leer[columnas]
   df = df[(df["Y"]>= -12.125)& (df["Y"]<= -11.625)]
   df = df[(df["X"]>= -76.125)& (df["X"]<= -75.625)]
   df = df[df["Control"]==0]
   df = df[(df["Polaridad"]== -3) | (df["Polaridad"]== 3)]
   archivos.append(df)
archivos

[      year  month  day  hour  minute  second         Y         X  Control  \
 89    2015      1    1    21      32      30 -11.73588 -76.01263        0   
 250   2015      1    2    19      52       1 -11.97078 -75.90048        0   
 378   2015      1    6    18      30      59 -11.81382 -75.99828        0   
 2855  2015      1   26    16      54      55 -11.91876 -75.70067        0   
 2871  2015      1   26    17      30      31 -11.83479 -76.01922        0   
 2879  2015      1   26    17      46       0 -11.78594 -75.79658        0   
 4153  2015      1   31    19      55      43 -11.65784 -76.03418        0   
 4163  2015      1   31    20       6      48 -11.65980 -76.05984        0   
 
       Polaridad  
 89           -3  
 250          -3  
 378          -3  
 2855          3  
 2871          3  
 2879          3  
 4153         -3  
 4163          3  ,
       year  month  day  hour  minute  second         Y         X  Control  \
 286   2015      2    4    20       5      29 

In [ ]:
combinaciones = []

# Realizar las combinaciones de los primeros 4 archivos
for i in range(0, 34, 2):
    comb = pd.merge(archivos[i], archivos[i+1], how="outer")
    combinaciones.append(comb)

# Realizar las combinaciones de las combinaciones anteriores
while len(combinaciones) > 1:
    new_combinaciones = []
    for i in range(0, len(combinaciones), 2):
        if i+1 < len(combinaciones):
            comb = pd.merge(combinaciones[i], combinaciones[i+1], how="outer")
            new_combinaciones.append(comb)
        else:
            # Agregar la combinación sin pareja (si hay un número impar de combinaciones)
            new_combinaciones.append(combinaciones[i])
    combinaciones = new_combinaciones

resultado_final = combinaciones[0]

In [ ]:
df_clim = pd.merge(resultado_final,archivos[34],how="outer")
df_clim.tail()

,year,month,day,hour,minute,second,Y,X,Control,Polaridad
2647,2017,11,29,19,57,45,-11.72079,-76.09520,0,-3
2648,2017,11,29,20,41,14,-11.81919,-76.07842,0,-3
2649,2017,11,29,21,23,24,-11.65403,-76.08765,0,-3
2650,2017,11,30,19,46,24,-11.69368,-75.79269,0,-3
2651,2017,11,30,19,56,38,-11.79359,-75.68331,0,3


In [ ]:
meses = ["enero", "febrero", "marzo","abril","mayo","junio","julio","agosto",
         "septiembre","octubre","noviembre","diciembre"]
df_por_mes = {}
for mes_numero, mes_nombre in enumerate(meses, start=1):
    df_por_mes[mes_nombre] = df_clim[df_clim["month"] == mes_numero]
for mes_nombre, df_por_mes in df_por_mes.items():
    globals()["df_" + mes_nombre] = df_por_mes

In [ ]:
lista_df = [df_enero,df_febrero,df_marzo,df_abril,df_mayo,df_junio,df_julio,
            df_agosto,df_septiembre,df_octubre,df_noviembre,df_diciembre]
lista_fechas = []
for i in lista_df:
  i["Fecha"]=pd.to_datetime(i[["year", "month", "day","hour","minute","second"]])
  i["light"]=1
  columnas=["Fecha","X","Y","light"]
  i=i[columnas]
  lista_fechas.append(i)

<ipython-input-10-f21c65e75798>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["Fecha"]=pd.to_datetime(i[["year", "month", "day","hour","minute","second"]])
<ipython-input-10-f21c65e75798>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["light"]=1
<ipython-input-10-f21c65e75798>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:
df_jan = lista_fechas[0]
df_feb = lista_fechas[1]
df_mar = lista_fechas[2]
df_apr = lista_fechas[3]
df_may = lista_fechas[4]
df_jun = lista_fechas[5]
df_jul = lista_fechas[6]
df_aug = lista_fechas[7]
df_sep = lista_fechas[8]
df_oct = lista_fechas[9]
df_nov = lista_fechas[10]
df_dic = lista_fechas[11]

## Por cuadriculas

In [ ]:
lista_aplicacion = [df_jan,df_feb,df_mar,df_apr,df_may,df_jun,df_jul,df_aug,
                    df_sep,df_oct,df_nov,df_dic]

clusters = []
for i in lista_aplicacion:
  #primer cuadrante: punto central (-76,-11.75)
  df_1 = i[(i["Y"]> -11.875)& (i["Y"]<= -11.625)]
  df_1 = df_1[(df_1["X"]> -76.125)& (df_1["X"]<= -75.875)]
  df_1 = df_1.set_index("Fecha")
  df1 = df_1.resample("60min").sum()
  df1["X"]= -76
  df1["Y"]= -11.75
  #segundo cuadrante: punto central (-75.75,-11.75)
  df_2 = i[(i["Y"]> -11.875)& (i["Y"]<= -11.625)]
  df_2 = df_2[(df_2["X"]> -75.875)& (df_2["X"]<= -75.625)]
  df_2 = df_2.set_index("Fecha")
  df2 = df_2.resample("60min").sum()
  df2["X"]= -75.75
  df2["Y"]= -11.75
  #tercer cuadrante: punto central (-76,-12)
  df_3 = i[(i["Y"]> -12.125)& (i["Y"]<= -11.875)]
  df_3 = df_3[(df_3["X"]> -76.125)& (df_3["X"]<= -75.875)]
  df_3 = df_3.set_index("Fecha")
  df3 = df_3.resample("60min").sum()
  df3["X"]= -76
  df3["Y"]= -12
  #cuarto cuadrante: punto central (-76,-11.75)
  df_4 = i[(i["Y"]> -12.125)& (i["Y"]<= -11.875)]
  df_4 = df_4[(df_4["X"]> -75.875)& (df_4["X"]<= -75.625)]
  df_4 = df_4.set_index("Fecha")
  df4 = df_4.resample("60min").sum()
  df4["X"]= -75.75
  df4["Y"]= -12

  #agrupando los cuadrantes
  df_total = pd.concat([df1,df2,df3,df4], axis=0).reset_index()
  df_total = df_total.sort_values(by = 'Fecha')
  df_total.rename(columns={"X":"longitude","Y":"latitude","Fecha":"time"},
                  inplace=True )
  #guardando
  clusters.append(df_total)

In [ ]:
enero_agrupado = clusters[0]
febrero_agrupado = clusters[1]
marzo_agrupado = clusters[2]
abril_agrupado = clusters[3]
mayo_agrupado = clusters[4]
junio_agrupado = clusters[5]
julio_agrupado = clusters[6]
agosto_agrupado = clusters[7]
septiembre_agrupado = clusters[8]
octubre_agrupado = clusters[9]
noviembre_agrupado = clusters[10]
diciembre_agrupado = clusters[11]

# Indice Cloud cover

In [ ]:
conv_inhibit_Junin = xr.open_dataset("Datasets/Convective_inhibition_Junin.nc")
conv_inhibit_prueba = conv_inhibit_Junin.sel(latitude=slice(-11.75,-12),
                             longitude=slice(-76,-75.75))
conv_inhibit_prueba

<xarray.Dataset>
Dimensions:    (longitude: 2, latitude: 2, time: 26304)
Coordinates:
  * longitude  (longitude) float32 -76.0 -75.75
  * latitude   (latitude) float32 -11.75 -12.0
  * time       (time) datetime64[ns] 2015-01-01 ... 2017-12-31T23:00:00
Data variables:
    cin        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-06-04 22:42:20 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
cin_enero = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 1)
cin_febrero = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 2)
cin_marzo = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 3)
cin_abril = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 4)
cin_mayo = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 5)
cin_junio = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 6)
cin_julio = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 7)
cin_agosto = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 8)
cin_sep = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 9)
cin_octubre = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 10)
cin_nov = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 11)
cin_dic = conv_inhibit_prueba.sel(time=conv_inhibit_prueba.time.dt.month == 12)

In [ ]:
#enero
cin_enero = cin_enero.to_dataframe()
df_cin_enero= cin_enero.reset_index()
#febrero
cin_febrero = cin_febrero.to_dataframe()
df_cin_febrero= cin_febrero.reset_index()
#marzo
cin_marzo = cin_marzo.to_dataframe()
df_cin_marzo= cin_marzo.reset_index()
#abril
cin_abril = cin_abril.to_dataframe()
df_cin_abril = cin_abril.reset_index()
#mayo
cin_mayo = cin_mayo.to_dataframe()
df_cin_mayo = cin_mayo.reset_index()
#junio
cin_junio = cin_junio.to_dataframe()
df_cin_junio = cin_junio.reset_index()
#julio
cin_julio = cin_julio.to_dataframe()
df_cin_julio= cin_julio.reset_index()
#agosto
cin_agosto = cin_agosto.to_dataframe()
df_cin_agosto = cin_agosto.reset_index()
#septiembre
cin_sep = cin_sep.to_dataframe()
df_cin_sep= cin_sep.reset_index()
#octubre
cin_octubre = cin_octubre.to_dataframe()
df_cin_octubre = cin_octubre.reset_index()
#noviembre
cin_nov = cin_nov.to_dataframe()
df_cin_nov= cin_nov.reset_index()
#diciembre
cin_dic = cin_dic.to_dataframe()
df_cin_dic = cin_dic.reset_index()

# Correlaciones considerando todos los casos

In [ ]:
#uniendo enero
df_corr_enero = pd.merge(enero_agrupado,df_cin_enero,how="outer")
df_corr_enero=df_corr_enero.dropna()
#uniendo febrero
df_corr_febrero = pd.merge(febrero_agrupado,df_cin_febrero,how="outer")
df_corr_febrero=df_corr_febrero.dropna()
#uniendo marzo
df_corr_marzo = pd.merge(marzo_agrupado,df_cin_marzo,how="outer")
df_corr_marzo=df_corr_marzo.dropna()
#uniendo abril
df_corr_abril = pd.merge(abril_agrupado,df_cin_abril,how="outer")
df_corr_abril= df_corr_abril.dropna()
#uniendo mayo
df_corr_mayo = pd.merge(mayo_agrupado,df_cin_mayo,how="outer")
#uniendo junio
df_corr_junio = pd.merge(junio_agrupado,df_cin_junio,how="outer")
df_corr_junio=df_corr_junio.dropna()
#uniendo julio
df_corr_julio = pd.merge(julio_agrupado,df_cin_julio,how="outer")
df_corr_julio=df_corr_julio.dropna()
#uniendo agosto
df_corr_agosto = pd.merge(agosto_agrupado,df_cin_agosto,how="outer")
df_corr_agosto=df_corr_agosto.dropna()
#uniendo septiembre
df_corr_septiembre = pd.merge(septiembre_agrupado,df_cin_sep,how="outer")
df_corr_septiembre=df_corr_septiembre.dropna()
#uniendo octubre
df_corr_octubre = pd.merge(octubre_agrupado,df_cin_octubre,how="outer")
#uniendo noviembre
df_corr_noviembre = pd.merge(noviembre_agrupado,df_cin_nov,how="outer")
df_corr_noviembre=df_corr_noviembre.dropna()
#uniendo diciembre
df_corr_diciembre = pd.merge(diciembre_agrupado,df_cin_dic,how="outer")
df_corr_diciembre=df_corr_diciembre.dropna()

## Correlaciones Spearman y Kendall: incluyendo dias sin rayos

In [ ]:
cin_enero_spearman = df_corr_enero['cin'].corr(df_corr_enero['light'], method='spearman')
cin_febrero_spearman = df_corr_febrero['cin'].corr(df_corr_febrero['light'], method='spearman')
cin_marzo_spearman = df_corr_marzo['cin'].corr(df_corr_marzo['light'], method='spearman')
cin_abril_spearman = df_corr_abril['cin'].corr(df_corr_abril['light'], method='spearman')
cin_mayo_spearman = df_corr_mayo['cin'].corr(df_corr_mayo['light'], method='spearman')
cin_junio_spearman = df_corr_junio['cin'].corr(df_corr_junio['light'], method='spearman')
cin_julio_spearman = df_corr_julio['cin'].corr(df_corr_julio['light'], method='spearman')
cin_agosto_spearman = df_corr_agosto['cin'].corr(df_corr_agosto['light'], method='spearman')
cin_septiembre_spearman = df_corr_septiembre['cin'].corr(df_corr_septiembre['light'], method='spearman')
cin_octubre_spearman = df_corr_octubre['cin'].corr(df_corr_octubre['light'], method='spearman')
cin_noviembre_spearman = df_corr_noviembre['cin'].corr(df_corr_noviembre['light'], method='spearman')
cin_diciembre_spearman = df_corr_diciembre['cin'].corr(df_corr_diciembre['light'], method='spearman')

In [ ]:
cin_enero_kendall = df_corr_enero['cin'].corr(df_corr_enero['light'], method='kendall')
cin_febrero_kendall = df_corr_febrero['cin'].corr(df_corr_febrero['light'], method='kendall')
cin_marzo_kendall = df_corr_marzo['cin'].corr(df_corr_marzo['light'], method='kendall')
cin_abril_kendall = df_corr_abril['cin'].corr(df_corr_abril['light'], method='kendall')
cin_mayo_kendall = df_corr_mayo['cin'].corr(df_corr_mayo['light'], method='kendall')
cin_junio_kendall = df_corr_junio['cin'].corr(df_corr_junio['light'], method='kendall')
cin_julio_kendall = df_corr_julio['cin'].corr(df_corr_julio['light'], method='kendall')
cin_agosto_kendall = df_corr_agosto['cin'].corr(df_corr_agosto['light'], method='kendall')
cin_septiembre_kendall = df_corr_septiembre['cin'].corr(df_corr_septiembre['light'], method='kendall')
cin_octubre_kendall = df_corr_octubre['cin'].corr(df_corr_octubre['light'], method='kendall')
cin_noviembre_kendall = df_corr_noviembre['cin'].corr(df_corr_noviembre['light'], method='kendall')
cin_diciembre_kendall = df_corr_diciembre['cin'].corr(df_corr_diciembre['light'], method='kendall')

In [ ]:
corr_spearman = [cin_enero_spearman, cin_febrero_spearman,cin_marzo_spearman,
                 cin_abril_spearman,cin_mayo_spearman,cin_junio_spearman,
                 cin_julio_spearman,cin_agosto_spearman,cin_septiembre_spearman,
                 cin_octubre_spearman,cin_noviembre_spearman,cin_diciembre_spearman]

In [ ]:
corr_kendall = [cin_enero_kendall, cin_febrero_kendall,cin_marzo_kendall,
                cin_abril_kendall,cin_mayo_kendall,cin_junio_kendall,
                cin_julio_kendall,cin_agosto_kendall,cin_septiembre_kendall,
                cin_octubre_kendall,cin_noviembre_kendall,cin_diciembre_kendall]

In [ ]:
data = {"Mes":[1,2,3,4,5,6,7,8,9,10,11,12],'Cin-Spearman': corr_spearman,
        'Cin-Kendall': corr_kendall}
df_correlaciones = pd.DataFrame(data)
df_correlaciones

,Mes,Cin-Spearman,Cin-Kendall
0,1,0.034748,0.028138
1,2,0.011482,0.009121
2,3,0.000886,0.000750
3,4,0.036537,0.029618
4,5,0.054312,0.044354
5,6,-0.092469,-0.075091
6,7,0.113228,0.094281
7,8,-0.090603,-0.073889
8,9,-0.041408,-0.033663
9,10,0.016252,0.013394


## Correlaciones Spearman y Kendall: no incluyendo dias sin rayos

In [ ]:
df_corr_enero1 = df_corr_enero[df_corr_enero["light"]>0]
df_corr_febrero1 = df_corr_febrero[df_corr_febrero["light"]>0]
df_corr_marzo1 = df_corr_marzo[df_corr_marzo["light"]>0]
df_corr_abril1 = df_corr_abril[df_corr_abril["light"]>0]
df_corr_mayo1 = df_corr_mayo[df_corr_mayo["light"]>0]
df_corr_junio1 = df_corr_junio[df_corr_junio["light"]>0]
df_corr_julio1 = df_corr_julio[df_corr_julio["light"]>0]
df_corr_agosto1 = df_corr_agosto[df_corr_agosto["light"]>0]
df_corr_septiembre1 = df_corr_septiembre[df_corr_septiembre["light"]>0]
df_corr_octubre1 = df_corr_octubre[df_corr_octubre["light"]>0]
df_corr_noviembre1 = df_corr_noviembre[df_corr_noviembre["light"]>0]
df_corr_diciembre1 = df_corr_diciembre[df_corr_diciembre["light"]>0]

In [ ]:
cin_enero_spearman1 = df_corr_enero1['cin'].corr(df_corr_enero1['light'], method='spearman')
cin_febrero_spearman1 = df_corr_febrero1['cin'].corr(df_corr_febrero1['light'], method='spearman')
cin_marzo_spearman1 = df_corr_marzo1['cin'].corr(df_corr_marzo1['light'], method='spearman')
cin_abril_spearman1 = df_corr_abril1['cin'].corr(df_corr_abril1['light'], method='spearman')
cin_mayo_spearman1 = df_corr_mayo1['cin'].corr(df_corr_mayo1['light'], method='spearman')
cin_junio_spearman1 = df_corr_junio1['cin'].corr(df_corr_junio1['light'], method='spearman')
cin_julio_spearman1 = df_corr_julio1['cin'].corr(df_corr_julio1['light'], method='spearman')
cin_agosto_spearman1 = df_corr_agosto1['cin'].corr(df_corr_agosto1['light'], method='spearman')
cin_septiembre_spearman1 = df_corr_septiembre1['cin'].corr(df_corr_septiembre1['light'], method='spearman')
cin_octubre_spearman1 = df_corr_octubre1['cin'].corr(df_corr_octubre1['light'], method='spearman')
cin_noviembre_spearman1 = df_corr_noviembre1['cin'].corr(df_corr_noviembre1['light'], method='spearman')
cin_diciembre_spearman1 = df_corr_diciembre1['cin'].corr(df_corr_diciembre1['light'], method='spearman')

In [ ]:
cin_enero_kendall1 = df_corr_enero1['cin'].corr(df_corr_enero1['light'], method='kendall')
cin_febrero_kendall1 = df_corr_febrero1['cin'].corr(df_corr_febrero1['light'], method='kendall')
cin_marzo_kendall1 = df_corr_marzo1['cin'].corr(df_corr_marzo1['light'], method='kendall')
cin_abril_kendall1 = df_corr_abril1['cin'].corr(df_corr_abril1['light'], method='kendall')
cin_mayo_kendall1 = df_corr_mayo1['cin'].corr(df_corr_mayo1['light'], method='kendall')
cin_junio_kendall1 = df_corr_junio1['cin'].corr(df_corr_junio1['light'], method='kendall')
cin_julio_kendall1 = df_corr_julio1['cin'].corr(df_corr_julio1['light'], method='kendall')
cin_agosto_kendall1 = df_corr_agosto1['cin'].corr(df_corr_agosto1['light'], method='kendall')
cin_septiembre_kendall1 = df_corr_septiembre1['cin'].corr(df_corr_septiembre1['light'], method='kendall')
cin_octubre_kendall1 = df_corr_octubre1['cin'].corr(df_corr_octubre1['light'], method='kendall')
cin_noviembre_kendall1 = df_corr_noviembre1['cin'].corr(df_corr_noviembre1['light'], method='kendall')
cin_diciembre_kendall1 = df_corr_diciembre1['cin'].corr(df_corr_diciembre1['light'], method='kendall')

In [ ]:
corr_spearman1 = [cin_enero_spearman1, cin_febrero_spearman1,cin_marzo_spearman1,
                 cin_abril_spearman1,cin_mayo_spearman1,cin_junio_spearman1,
                 cin_julio_spearman1,cin_agosto_spearman1,cin_septiembre_spearman1,
                 cin_octubre_spearman1,cin_noviembre_spearman1,cin_diciembre_spearman1]

corr_kendall1 = [cin_enero_kendall1, cin_febrero_kendall1,cin_marzo_kendall1,
                cin_abril_kendall1,cin_mayo_kendall1,cin_junio_kendall1,
                cin_julio_kendall1,cin_agosto_kendall1,cin_septiembre_kendall1,
                cin_octubre_kendall1,cin_noviembre_kendall1,cin_diciembre_kendall1]

In [ ]:
data1 = {"Mes":[1,2,3,4,5,6,7,8,9,10,11,12],'Cin-Spearman_1': corr_spearman1,
        'Cin-Kendall_1': corr_kendall1}
df_correlaciones1 = pd.DataFrame(data1)
df_correlaciones1

,Mes,Cin-Spearman_1,Cin-Kendall_1
0,1,0.165287,0.123437
1,2,-0.024007,-0.020829
2,3,0.090689,0.066944
3,4,-0.206440,-0.159170
4,5,0.181650,0.158114
5,6,-0.036827,-0.077152
6,7,NaN,NaN
7,8,-0.040452,-0.030934
8,9,0.000000,0.000000
9,10,0.093506,0.085542


In [ ]:
cin_dataframe = pd.merge(df_correlaciones,df_correlaciones1,how="outer")
cin_dataframe

,Mes,Cin-Spearman,Cin-Kendall,Cin-Spearman_1,Cin-Kendall_1
0,1,0.034748,0.028138,0.165287,0.123437
1,2,0.011482,0.009121,-0.024007,-0.020829
2,3,0.000886,0.000750,0.090689,0.066944
3,4,0.036537,0.029618,-0.206440,-0.159170
4,5,0.054312,0.044354,0.181650,0.158114
5,6,-0.092469,-0.075091,-0.036827,-0.077152
6,7,0.113228,0.094281,NaN,NaN
7,8,-0.090603,-0.073889,-0.040452,-0.030934
8,9,-0.041408,-0.033663,0.000000,0.000000
9,10,0.016252,0.013394,0.093506,0.085542


# Guardando

In [ ]:
df_cin_correlaciones = pd.concat([df_corr_enero,df_corr_febrero,df_corr_marzo,
                                   df_corr_abril,df_corr_mayo,df_corr_junio,
                                   df_corr_julio, df_corr_agosto,df_corr_septiembre,
                                   df_corr_octubre, df_corr_noviembre, df_corr_diciembre])
df_cin_correlaciones

,time,longitude,latitude,light,cin
0,2015-01-01 21:00:00,-76.00,-11.75,1.0,1.706421
1,2015-01-01 22:00:00,-76.00,-11.75,0.0,0.030457
2,2015-01-01 23:00:00,-76.00,-11.75,0.0,16.470245
3,2015-01-02 00:00:00,-76.00,-11.75,0.0,1.386475
4,2015-01-02 01:00:00,-76.00,-11.75,0.0,6.155365
...,...,...,...,...,...
37378,2016-12-30 19:00:00,-75.75,-11.75,0.0,21.467682
37379,2016-12-30 19:00:00,-76.00,-11.75,1.0,35.545868
37380,2016-12-30 20:00:00,-75.75,-11.75,0.0,23.539795
37382,2016-12-30 21:00:00,-76.00,-11.75,1.0,28.582947


In [ ]:
df_cin_correlaciones.to_csv("Scikit_regresion_multivariada/Convective_inhibition.csv")

In [ ]:
cin_dataframe.to_csv("Correlaciones_rayos/Correlacion_Cin.csv")